**Percobaan 1**

In [ ]:
# jalankan ini di Colab
!apt-get update -qq
!pip install PyPDF2==3.0.1 reportlab Unidecode --quiet


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 17.1 MB/s eta 0:00:00


In [ ]:
# -----------------------
# Imports & helper
# -----------------------
import time, json, base64
from pathlib import Path
from collections import Counter
from typing import List, Tuple, Dict
from PyPDF2 import PdfReader
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from unidecode import unidecode
import math


In [ ]:
# -----------------------
# Colab uploader (jika perlu)
# -----------------------
def upload_files():
    try:
        from google.colab import files
    except Exception as e:
        raise RuntimeError("Fungsi upload hanya berjalan di Google Colab.") from e
    uploaded = files.upload()
    for name in uploaded.keys():
        print(f"Uploaded: {name}")
    return list(uploaded.keys())

In [ ]:
# -----------------------
# Utility: ASCII check & extraction
# -----------------------
def has_non_ascii(s: str) -> bool:
    return any(ord(ch) >= 128 for ch in s)

def extract_text_and_force_ascii(path: str, transliterate: bool=True) -> str:
    """
    1) Extract text dari PDF (PyPDF2).
    2) Jika ditemukan karakter non-ASCII:
       - jika transliterate=True: lakukan unidecode -> hasil ASCII (lossy).
       - jika transliterate=False: raise ValueError .
    Mengembalikan string ASCII-only.
    """
    reader = PdfReader(path)
    pages = []
    for p in reader.pages:
        # extract_text mungkin None jika page kosong
        pages.append(p.extract_text() or "")
    text_all = "\n".join(pages)

    if not text_all.strip():
        # Jika kosong kemungkinan scan/image; proposal mengharuskan PDF berbasis teks.
        raise ValueError("Ekstraksi teks kosong — kemungkinan PDF adalah hasil scan. Proposal mengharuskan PDF berbasis teks ASCII.")

    if has_non_ascii(text_all):
        if transliterate:
            print("Ditemukan karakter non-ASCII. Melakukan transliteration ke ASCII menggunakan Unidecode.")
            text_trans = unidecode(text_all)
            # safety: pastikan hanya ASCII
            text_ascii = text_trans.encode('ascii', errors='ignore').decode('ascii')
            if has_non_ascii(text_ascii):
                # seharusnya tidak terjadi karena kita encode-decode
                raise ValueError("Transliteration gagal menghapus semua non-ASCII.")
            return text_ascii
        else:
            raise ValueError("Terdeteksi karakter non-ASCII. Proposal membatasi ke ASCII.")
    return text_all

In [ ]:
# -----------------------
# Caesar & Vigenere (ASCII range 0..127)
# -----------------------
def caesar_encrypt(text: str, shift: int) -> str:
    out = []
    for ch in text:
        code = ord(ch)
        if 0 <= code < 128:
            out.append(chr((code + shift) % 128))
        else:
            out.append(ch)  # seharusnya tidak terjadi karena ASCII-only
    return "".join(out)

def caesar_decrypt(text: str, shift: int) -> str:
    return caesar_encrypt(text, -shift)

def vigenere_encrypt(text: str, key: str) -> str:
    key_bytes = [ord(k) % 128 for k in key]
    out = []
    ki = 0
    for c in text:
        cb = ord(c)
        if 0 <= cb < 128:
            kb = key_bytes[ki % len(key_bytes)]
            out.append(chr((cb + kb) % 128))
            ki += 1
        else:
            out.append(c)
    return "".join(out)

def vigenere_decrypt(text: str, key: str) -> str:
    key_bytes = [ord(k) % 128 for k in key]
    out = []
    ki = 0
    for c in text:
        cb = ord(c)
        if 0 <= cb < 128:
            kb = key_bytes[ki % len(key_bytes)]
            out.append(chr((cb - kb) % 128))
            ki += 1
        else:
            out.append(c)
    return "".join(out)

In [ ]:
# -----------------------
# Columnar transposition
# -----------------------
def columnar_transpose_encrypt(text: str, n_cols: int, col_order: List[int], pad_char: str='X') -> Tuple[str,int]:
    if n_cols <= 0:
        raise ValueError("n_cols harus > 0")
    L = len(text)
    pad_needed = (n_cols - (L % n_cols)) % n_cols
    text_padded = text + (pad_char * pad_needed)
    rows = len(text_padded) // n_cols
    matrix = [list(text_padded[i*n_cols:(i+1)*n_cols]) for i in range(rows)]
    # normalisasi col_order: jika 1-based ubah ke 0-based
    order0 = [o-1 if min(col_order) >= 1 else o for o in col_order]
    if sorted(order0) != list(range(n_cols)):
        raise ValueError("col_order tidak valid untuk n_cols.")
    out = []
    for c in order0:
        for r in range(rows):
            out.append(matrix[r][c])
    return "".join(out), pad_needed

def columnar_transpose_decrypt(cipher: str, n_cols: int, col_order: List[int], pad_needed: int, pad_char: str='X') -> str:
    L = len(cipher)
    if n_cols <= 0:
        raise ValueError("n_cols harus > 0")
    rows = L // n_cols
    order0 = [o-1 if min(col_order) >= 1 else o for o in col_order]
    cols = []
    idx = 0
    for _ in range(n_cols):
        cols.append(list(cipher[idx:idx+rows]))
        idx += rows
    matrix = [[''] * n_cols for _ in range(rows)]
    for read_pos, c in enumerate(order0):
        for r in range(rows):
            matrix[r][c] = cols[read_pos][r]
    flat = "".join("".join(row) for row in matrix)
    if pad_needed:
        flat = flat[:-pad_needed]
    return flat


In [ ]:
# -----------------------
# Metrics: entropy & IC
# -----------------------
def entropy(text: str) -> float:
    N = len(text)
    if N == 0: return 0.0
    freqs = Counter(text)
    ent = -sum((f/N) * math.log2(f/N) for f in freqs.values())
    return ent

def index_of_coincidence(text: str) -> float:
    N = len(text)
    if N <= 1: return 0.0
    freqs = Counter(text)
    num = sum(f*(f-1) for f in freqs.values())
    den = N*(N-1)
    return num/den


In [ ]:
# -----------------------
# Write text ke PDF (monospace)
# -----------------------
def text_to_pdf(text: str, out_path: str, page_width=595, page_height=842, font_size=10, margin=40):
    c = canvas.Canvas(out_path, pagesize=(page_width, page_height))
    lines = text.split("\n")
    x = margin
    y = page_height - margin
    line_height = font_size * 1.2
    c.setFont("Courier", font_size)
    for line in lines:
        if y < margin + line_height:
            c.showPage()
            c.setFont("Courier", font_size)
            y = page_height - margin
        max_chars = int((page_width - 2*margin) / (font_size * 0.6))
        while len(line) > max_chars:
            c.drawString(x, y, line[:max_chars])
            line = line[max_chars:]
            y -= line_height
            if y < margin + line_height:
                c.showPage()
                c.setFont("Courier", font_size)
                y = page_height - margin
        c.drawString(x, y, line)
        y -= line_height
    c.save()


In [ ]:
# -----------------------
# Encrypt & Decrypt (full) — sesuai proposal dengan penyimpanan Base64
# -----------------------
def encrypt_pdf_hybrid_ascii(input_pdf: str, out_cipher_pdf: str,
                       shift: int, vigenere_key: str,
                       n_cols: int, col_order: List[int],
                       pad_char: str='X', transliterate_nonascii: bool=True) -> Dict:
    """
    Enkripsi: extract plaintext (ASCII atau transliterate jika diizinkan), Caesar -> Vigenere -> Columnar.
    Simpan ciphertext ke PDF sebagai Base64 (ASCII-safe).
    Kembalikan metadata dict (dan simpan .meta.json).
    """
    plaintext = extract_text_and_force_ascii(input_pdf, transliterate=transliterate_nonascii)
    result = {"input_len": len(plaintext)}
    # layer 1: Caesar
    t0 = time.perf_counter()
    caes = caesar_encrypt(plaintext, shift)
    t1 = time.perf_counter()
    # layer 2: Vigenere
    vig = vigenere_encrypt(caes, vigenere_key)
    t2 = time.perf_counter()
    # layer 3: Columnar transposition
    cipher_text, pad_needed = columnar_transpose_encrypt(vig, n_cols, col_order, pad_char=pad_char)
    t3 = time.perf_counter()

    # encode ciphertext ke Base64 sehingga aman ditulis & diekstrak dari PDF
    cipher_b64 = base64.b64encode(cipher_text.encode('latin-1')).decode('ascii')

    # tulis Base64 ke PDF
    text_to_pdf(cipher_b64, out_cipher_pdf)

    # metadata file (sesuai proposal)
    meta = {
        "scheme": "Caesar+Vigenere+Columnar (ASCII), stored as Base64 in PDF",
        "shift": shift,
        "vigenere_key": vigenere_key,
        "n_cols": n_cols,
        "col_order": col_order,
        "pad_char": pad_char,
        "pad_needed": pad_needed,
        "input_file": str(Path(input_pdf).name),
        "cipher_file": str(Path(out_cipher_pdf).name),
        "stored_base64": True,
        "metrics": {
            "entropy_plaintext": entropy(plaintext),
            "entropy_cipher": entropy(cipher_text),
            "ic_plaintext": index_of_coincidence(plaintext),
            "ic_cipher": index_of_coincidence(cipher_text)
        },
        "timings": {
            "caesar": t1 - t0,
            "vigenere": t2 - t1,
            "transposition": t3 - t2,
            "total": t3 - t0
        }
    }
    meta_path = str(Path(out_cipher_pdf).with_suffix('.meta.json'))
    with open(meta_path, 'w', encoding='utf-8') as f:
        json.dump(meta, f, indent=2)
    print(f"Cipher PDF written to: {out_cipher_pdf}")
    print(f"Meta written to: {meta_path}")
    return meta

def decrypt_pdf_hybrid_ascii(cipher_pdf: str, meta_path: str, out_decoded_pdf: str) -> Dict:
    """
    Dekripsi: ekstrak teks PDF (harus Base64 ASCII), decode Base64 -> ciphertext asli,
    lalu inverse columnar -> inverse vigenere -> inverse caesar.
    Menulis plaintext ke PDF dan mengembalikan plaintext juga untuk verifikasi.
    """
    cipher_b64 = extract_text_and_force_ascii(cipher_pdf, transliterate=False)

    with open(meta_path, 'r', encoding='utf-8') as f:
        meta = json.load(f)

    if not meta.get('stored_base64', False):
        raise ValueError("Meta file tidak menyatakan bahwa ciphertext disimpan sebagai Base64. Tidak cocok dengan format yang diharapkan.")

    try:
        cipher_bytes = base64.b64decode(cipher_b64.encode('ascii'))
        cipher_text = cipher_bytes.decode('latin-1')
    except Exception as e:
        raise ValueError("Gagal decode Base64 dari PDF cipher. Isi PDF mungkin terpotong atau tidak valid.") from e

    shift = meta['shift']
    vigenere_key = meta['vigenere_key']
    n_cols = meta['n_cols']
    col_order = meta['col_order']
    pad_needed = meta.get('pad_needed', 0)

    # inverse langkah-langkah
    t0 = time.perf_counter()
    pre_vig = columnar_transpose_decrypt(cipher_text, n_cols, col_order, pad_needed, pad_char=meta.get('pad_char','X'))
    t1 = time.perf_counter()
    pre_caes = vigenere_decrypt(pre_vig, vigenere_key)
    t2 = time.perf_counter()
    plaintext = caesar_decrypt(pre_caes, shift)
    t3 = time.perf_counter()

    # pastikan ASCII-only sesuai proposal
    if has_non_ascii(plaintext):
        raise ValueError("Hasil dekripsi mengandung karakter non-ASCII — verifikasi kunci/parameter diperlukan.")

    # tulis hasil dekripsi ke PDF
    text_to_pdf(plaintext, out_decoded_pdf)

    timings = {"inv_transposition": t1-t0, "inv_vigenere": t2-t1, "inv_caesar": t3-t2, "total": t3-t0}
    print(f"Decrypted PDF written to: {out_decoded_pdf}")
    return {"timings": timings, "plaintext_len": len(plaintext), "plaintext": plaintext}

In [ ]:
# -----------------------
# Contoh pemakaian (run di Colab)
# -----------------------
if __name__ == "__main__":
    print("Jika ingin mencoba, upload PDF teks (bukan hasil scan).")
    try:
        files = upload_files()  # unggah file via Colab UI
    except RuntimeError as e:
        print(str(e))
        # Jika bukan Colab, user bisa set langsung path file manual
        raise

    input_pdf_path = files[0]
    print(f"Input PDF: {input_pdf_path}")

    # Parameter contoh (ubah sesuai kebutuhan/proposal)
    out_cipher_pdf = "cipher_output.pdf"
    # out_decoded_pdf = "decoded_output.pdf"
    shift = 5
    vigenere_key = "KUNCI"
    n_cols = 6
    col_order = [3,1,6,2,5,4]
    pad_char = 'X'

    # ENKRIPSI (transliterate_nonascii=True jika ingin otomatis konversi karakter non-ASCII -> ASCII)
    try:
        meta = encrypt_pdf_hybrid_ascii(input_pdf_path, out_cipher_pdf,
                                        shift=shift, vigenere_key=vigenere_key,
                                        n_cols=n_cols, col_order=col_order,
                                        pad_char=pad_char, transliterate_nonascii=True)
        print(json.dumps(meta, indent=2))
    except Exception as e:
        print("Enkripsi gagal:", str(e))
        raise

    # DEKRIPSI
    # try:
    #     dec = decrypt_pdf_hybrid_ascii(out_cipher_pdf, out_cipher_pdf.replace('.pdf', '.meta.json'), out_decoded_pdf)
    #     print("Hasil dekripsi timings & len:", dec["timings"], dec["plaintext_len"])
    # except Exception as e:
    #     print("Dekripsi gagal:", str(e))
    #     raise

    # Verifikasi: bandingkan plaintext asli (setelah transliteration saat enkripsi) dengan hasil dekripsi
    try:
        # Ambil plaintext asli yang digunakan enkripsi dari file input (gunakan transliterate=True karena enkripsi menggunakan transliterate=True)
        original_plaintext = extract_text_and_force_ascii(input_pdf_path, transliterate=True)
        decrypted_plaintext = dec["plaintext"]
        same = (original_plaintext == decrypted_plaintext)
        print("Verifikasi plaintext == decrypted_plaintext ?", same)
        if not same:
            # tampilkan preview perbedaan awal (first 400 char)
            print("Preview original (start):")
            print(original_plaintext[:400])
            print("Preview decrypted (start):")
            print(decrypted_plaintext[:400])
    except Exception as e:
        print("Verifikasi gagal:", str(e))

    print("Selesai. Files created:", out_cipher_pdf, out_cipher_pdf.replace('.pdf', '.meta.json'), out_decoded_pdf)

Jika ingin mencoba, upload PDF teks (bukan hasil scan).


Saving Percobaan 1.pdf to Percobaan 1.pdf
Uploaded: Percobaan 1.pdf
Input PDF: Percobaan 1.pdf
Ditemukan karakter non-ASCII. Melakukan transliteration ke ASCII menggunakan Unidecode.
Cipher PDF written to: cipher_output.pdf
Meta written to: cipher_output.meta.json
{
  "scheme": "Caesar+Vigenere+Columnar (ASCII), stored as Base64 in PDF",
  "shift": 5,
  "vigenere_key": "KUNCI",
  "n_cols": 6,
  "col_order": [
    3,
    1,
    6,
    2,
    5,
    4
  ],
  "pad_char": "X",
  "pad_needed": 4,
  "input_file": "Percobaan 1.pdf",
  "cipher_file": "cipher_output.pdf",
  "stored_base64": true,
  "metrics": {
    "entropy_plaintext": 4.206534931463318,
    "entropy_cipher": 5.458059282188951,
    "ic_plaintext": 0.07909918370660662,
    "ic_cipher": 0.02779736247536334
  },
  "timings": {
    "caesar": 0.0009123920001457009,
    "vigenere": 0.0012859129999469587,
    "transposition": 0.0008148029999119899,
    "total": 0.0030131080000046495
  }
}
Ditemukan karakter non-ASCII. Melakukan transl

In [ ]:

    try:
        dec = decrypt_pdf_hybrid_ascii(out_cipher_pdf, out_cipher_pdf.replace('.pdf', '.meta.json'), out_decoded_pdf)
        print("Hasil dekripsi timings & len:", dec["timings"], dec["plaintext_len"])
    except Exception as e:
        print("Dekripsi gagal:", str(e))
        raise
    out_decoded_pdf = "decoded_output.pdf"

Decrypted PDF written to: decoded_output.pdf
Hasil dekripsi timings & len: {'inv_transposition': 0.0005198149999614543, 'inv_vigenere': 0.0007817220000561065, 'inv_caesar': 0.00048034500014182413, 'total': 0.001781882000159385} 3668
